### このノート

### memo
等式制約、漸近論を使っていないから良い、nealの設定で1, -1に変えられるか、十分統計量、t^はオリジナルのデータを使って計算した、漸近論はnをすごく増やしたときにする、qaで統計の問題を解くのはなかなかない、戦前（ビッグデータがない）に作った基準を今も使っている、最尤推定をする際に十分統計量だけ知っていれば圧縮な可能

### 方法
```
for 0（特徴量なし）→j（全特徴量-1）回 :
    for 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいxsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量の数 = 調べる特徴量の数 - 1　（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ


### nealを使う


In [2]:
#!pip3 install dwave-neal

In [3]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo

In [4]:
X_mushroom = pd.read_csv("../input/X_mushroom.csv", sep=',', index_col=0)
y_mushroom = pd.read_csv('../input/y_mushroom.csv', sep=',', index_col=0)['21']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

In [21]:
#columns #'0', '1',..,'118'

In [5]:
num_samples

8124

In [6]:
y_sum

4208

In [7]:
# BINARY変数
y = Array.create('y', shape=num_samples, vartype='BINARY')
#y2 = Array.create('y', shape=5, vartype='BINARY')
y2_sum = 2

In [19]:
def H(minP_cols):
    
    #QUBO式で定式化
    H = (sum(y) - y_sum)**2
    　
    for i in minP_cols:
        H_plus = (ti_dic[np.dot(X_mushroom[i], y)] - ti_dic[np.dot(X_mushroom[i], y_mushroom)])**2
        H += H_plus
        
    return H

SyntaxError: unexpected EOF while parsing (<ipython-input-19-d810ccb98a80>, line 1)

In [ ]:
def make_y(H):
    # モデルをコンパイル
    model = H.compile()

    # プレースホルダーに具体的な数値を当てはめて、QUBOを作成
    feed_dict = {"cover": 1.0}
    qubo, offset = model.to_qubo(feed_dict=feed_dict)

    # SAで解を探索する
    raw_solution = solve_qubo(qubo)

    # 得られた結果をデコードする
    decoded_solution, broken, energy = model.decode_solution(raw_solution, vartype="BINARY", feed_dict=feed_dict)

    return pd.Series(list(decoded_solution['y'].values()))

### 方法
```
while 調べる特徴量がある限り:
    for 調べる特徴量 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいxsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量の数 = 調べる特徴量の数 - 1　（更新する）
```


In [14]:
minP_cols = {}
servey_num_col_list = columns

while len(servey_num_col_list) > 0:
    
    p = {}
    for i in servey_num_col_list:
        ori_ti = np.dot(X_mushroom[i], y_mushroom) #特徴量iを表すp
        
        ti_dic = {} #分布の要素となる値を格納する
        for _ in range(100):   
            H = H(minP_cols)
            y = make_y(H)
            ti_dic[np.dot(X_mushroom[i], y)] += 1
        bunshi = sum([ti_dic[ii] for ii in range(int(ori_ti), int(list(ti_dic.keys)[-1])+1)])
        p[i] = bunshi/sum(list(ti_dic.values))
        
    minP_keys = [k for k, v in p.items() if v == min(list(p.values))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minp_keys]

SyntaxError: invalid syntax (<ipython-input-14-17ec753b7213>, line 8)